# Исследование надёжности заёмщиков

Определить влияние семейного положения, дохода и количества детей клиента на факт погашения кредита в срок.

Входные данные от банка — статистика о платёжеспособности клиентов.


# Оглавление
- [Шаг 1. Откроем файл с данными и изучим общую информацию.](#1)
- [Шаг 2. Предобработка данных](#2)
    - [Лемматизация](#2.1)
    - [Категоризация данных](#2.2)
- [Шаг 3. Что влияет на возврат кредита](#3)
    - [Зависимость между наличием детей и возвратом кредита](#3.1)
    - [Влияние семейного положения на возврат кредита ](#3.2)
    - [Зависимость между уровнем дохода и возвратом кредита](#3.3)
    - [Влияние целей кредита на его возврат ](#3.4)
- [ Шаг 4. Общий вывод](#4)


### Шаг 1. Откройте файл с данными и изучите общую информацию.  <a class="anchor" id="1"></a>

In [1]:
import pandas as pd
data=pd.read_csv('/datasets/data.csv')
print(data.info())
print('Количество полных дубликатов', data.duplicated().sum())
print (data['children'].unique())
print ('отрицательных детей', data[(data['children']<0)]['children'].count())
print ('детей больше 15', data[data['children']>=15]['children'].count())

print(data.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
Количество полных дубликатов 54
[ 1  0  3  2 -1  4 20  5]
отрицательных детей 47
детей больше 15 76
    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее          

### Вывод

- У всех, кроме пенсионеров отрицательный трудовой стаж, а у пенсионеров стаж неправдоподобный(по 1000лет), возможно он указан в пересчете на часы? Похоже на техническую ошибку
- Пропуски типа NaN в стаже и доходе (причем по количеству пропуски в доходе равны пропускам в стаже, вероятно это одни и те же записи) - около 10%, слишком много, чтобы просто выкинуть эти строки. Вероятно пропуски неслучайные, сами респонденты не захотели отвечать на вопрос о доходе и трудовом стаже.
- разный регистр в семейном статусе и образовании, 
- возраст указан как тип object (строка), а не число. 
- 54 полных дубликата. поскольку нет никакого однозначного идентификатора клиент (например, имени), то это могут быть разные люди с одинаковыми вводными данными, тем более что наиболее характерные параметры, такие как доход и стаж, у дубликатов отсутствуют, поэтому я бы их не удаляла из анализа. 
Другая возможная причина возникновения дубликато - один клиент мог брать два и более кредита. 
В любом случае  54 дубликата - это меньше 1% данных, поэтому мы много не потеряем даже если удалим дубликаты:)
- количество детей отрицательное и 20 - в сумме таких записей около 100. можно удалить эти записи 


### Шаг 2. Предобработка данных <a class="anchor" id="2"></a>

### Обработка пропусков

In [2]:

data.loc[data['days_employed']<0, 'days_employed']*=-1 #делаем отрицательный стаж положительным
data.loc[data['income_type']=='пенсионер', 'days_employed']=data['days_employed']/24 #переводим часы в дни
data = data[data['children']>=0] #убираем некорректные значения по количеству детей
data = data[data['children']<=15]
print(data['children'].unique())
#print(data.head(10))


#заменяем пропуски в стаже на среднее значение стажа для каждого возраста
means = data.groupby(['dob_years'])['days_employed'].mean()
data = data.set_index(['dob_years'])
data['days_employed'] = data['days_employed'].fillna(means)
data = data.reset_index()

#заменяем пропуски в доходе по типу занятости
means = data.groupby(['income_type'])['total_income'].mean()
data = data.set_index(['income_type'])
data['total_income'] = data['total_income'].fillna(means)
data = data.reset_index()


#проверка
print(data.head(50))
print(data.info())        


[1 0 3 2 4 5]
    income_type  dob_years  children  days_employed            education  \
0     сотрудник         42         1    8437.673028               высшее   
1     сотрудник         36         1    4024.803754              среднее   
2     сотрудник         33         0    5623.422610              Среднее   
3     сотрудник         32         3    4124.747207              среднее   
4     пенсионер         53         0   14177.753002              среднее   
5     компаньон         27         0     926.185831               высшее   
6     компаньон         43         0    2879.202052               высшее   
7     сотрудник         50         0     152.779569              СРЕДНЕЕ   
8     сотрудник         35         2    6929.865299               ВЫСШЕЕ   
9     сотрудник         41         0    2188.756445              среднее   
10    компаньон         36         2    4171.483647               высшее   
11    сотрудник         40         0     792.701887              среднее  

### Вывод

1) Необходимо сначала разобраться с артефактами. 
    Отрицательные значение сделать положительными. 
    Значения для пенсионеров привести ко дням (если гипотеза, что данные в часах верна)

2) Данные пропущены в столбцах days_employed и total_income

    -days_employed(стаж) больше всего зависит от возраста, поэтому заменяем пропуски в стаже на среднее значение стажа для каждого возраста

    -total_income(доход) в большей степени зависит от income_type (типа занятости), заменяем пропуски в доходе на среднее значение для данного типа занятости
    

### Замена типа данных

In [3]:
data['days_employed']=data['days_employed'].astype(int)
data['dob_years']=data['dob_years'].astype(int)
#проверка
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 12 columns):
income_type         21402 non-null object
dob_years           21402 non-null int64
children            21402 non-null int64
days_employed       21402 non-null int64
education           21402 non-null object
education_id        21402 non-null int64
family_status       21402 non-null object
family_status_id    21402 non-null int64
gender              21402 non-null object
debt                21402 non-null int64
total_income        21402 non-null float64
purpose             21402 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
None


### Вывод

приводим количество дней в стаже и возраст к целочисленному значению. поскольку метод to_numeric() переводит в тип float, то используем astype(). total_income оставляем с плавающей точкой

### Обработка дубликатов

In [4]:
#приводим все текстовые строки к нижнему регистру
data['education']=data['education'].str.lower()
data['income_type']=data['income_type'].str.lower()
data['family_status']=data['family_status'].str.lower()
data['gender']=data['gender'].str.lower()
#ищем дубликаты
print('количество дубликатов', data.duplicated().sum())
data=data.drop_duplicates()
#проверка
print('количество дубликатов', data.duplicated().sum())
print(data.info())


количество дубликатов 71
количество дубликатов 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21331 entries, 0 to 21401
Data columns (total 12 columns):
income_type         21331 non-null object
dob_years           21331 non-null int64
children            21331 non-null int64
days_employed       21331 non-null int64
education           21331 non-null object
education_id        21331 non-null int64
family_status       21331 non-null object
family_status_id    21331 non-null int64
gender              21331 non-null object
debt                21331 non-null int64
total_income        21331 non-null float64
purpose             21331 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.1+ MB
None


### Вывод

-чтобы учесть все дубликаты переводим текстовые значения в нижний регистр.

-затем убираем полные дубликаты строк методом drop_duplicates() и  обновляем индексацию.

-ищем дубликаты методом duplicated(), поскольку метод values_counts() применяют к объекту Series, а нам надо проанализировать датафрейм по врем столбцам

-дубликаты могли возникнуть как из-за технической ошибки, так и из-за отсутствия однозначного идентификатора клиента(имени, телефона и тп). кроме того один и тот же человек мог несколько раз брать кредиты на разные цели


### Лемматизация <a class="anchor" id="2.1"></a>

In [5]:
from pymystem3 import Mystem
m = Mystem()
      
def lemat(str):
        lemmas=m.lemmatize(str)
        for word in lemmas:
            if word == 'образование':
                return 'образование'
            if word == 'автомобиль':
                return 'автомобиль'
            if word == 'недвижимость':
                return 'недвижимость'
            if word == 'жилье':
                return 'недвижимость'
            if word == 'свадьба':
                return 'свадьба'
        return 'проверь цель' 

data['main_purpose']=data['purpose'].apply(lemat)                                
print(data.head(20))
print(data['main_purpose'].value_counts())


   income_type  dob_years  children  days_employed            education  \
0    сотрудник         42         1           8437               высшее   
1    сотрудник         36         1           4024              среднее   
2    сотрудник         33         0           5623              среднее   
3    сотрудник         32         3           4124              среднее   
4    пенсионер         53         0          14177              среднее   
5    компаньон         27         0            926               высшее   
6    компаньон         43         0           2879               высшее   
7    сотрудник         50         0            152              среднее   
8    сотрудник         35         2           6929               высшее   
9    сотрудник         41         0           2188              среднее   
10   компаньон         36         2           4171               высшее   
11   сотрудник         40         0            792              среднее   
12   пенсионер         65

### Вывод

Мы выделили леммы, а затем в отдельный столбец добавили унифицированные названия целей. Названия выделили на основе визуального анализа таблицы. Поскольку пропусков в новом столбце не возникло, то делаем вывод, что все цели распознаны верно.

### Категоризация данных <a class="anchor" id="2.2"></a>

In [44]:
#категоризация по уровню дохода
# поделим доход на 6 групп
#меньше 50К - доход "А", 50К-80K- доход "А+", 80K-110K-"B",  150-200К - доход "С"，больше 200К - доход "С+". Остальные "В+"

print('минимальный доход: {:.0f}'.format(data['total_income'].min()),
      '\nмаксимальный доход: {:.0f}'. format(data['total_income'].max()),
      '\nсредний доход заемщика: {:.0f}'.format(data['total_income'].mean()), 
      '\nмедианный доход заемщика: {:.0f}'.format(data['total_income'].median()))

def income_group(income):
    if income<50000:
        return 'A'
    if 50000<=income<80000:
        return 'A+'
    if 80000<=income<110000:
        return 'B'     
    if 200000>income>=150000:
        return 'C'
    if income>150000:
        return 'C+'
    return 'B+'

def childen_group(children):
    if children>=1:
        return 'Y'
    return 'N'

data['income_group']=data['total_income'].apply(income_group)
data['children_group']=data['children'].apply(childen_group)
data.head(10)
print(data.groupby(by='income_group')['total_income'].count())
print(data.groupby(by='children_group')['children_group'].count())

минимальный доход: 20667 
максимальный доход: 2265604 
средний доход заемщика: 167458 
медианный доход заемщика: 151905
income_group
A      371
A+    1894
B     3352
B+    4871
C     5307
C+    5536
Name: total_income, dtype: int64
children_group
N    14091
Y     7240
Name: children_group, dtype: int64


### Вывод

## Шаг 3. Что влияет на возврат кредита <a class="anchor" id="3"></a>

### Зависимость между наличием детей и возвратом кредита <a class="anchor" id="3.1"></a>

In [84]:
#вычислим средний по заемщикам процент задолженностей
average_percent=data['debt'].sum()/data['debt'].count()
#создадим таблицу зависимости между наличием детей и наличием задоженности по кредиту
data_children=data.pivot_table(index=['children','children_group'], columns='debt', values='gender', aggfunc='count').reset_index()

#вычисляем процент должников среди каждой группы
data_children['percent_of_debt']=data_children[1]/(data_children[1]+data_children[0])

#вычисляем аффинити должников по каждой группе
data_children['affinity_of_debt']=data_children['percent_of_debt']/average_percent

#вычисляем процент должников и аффинити среди имеющих детей в целом
data_with_children=data_children[data_children['children_group']=='Y']
per_debt_with_children=data_with_children[1].sum()/(data_with_children[0].sum()+data_with_children[1].sum())
aff_debt_with_children = per_debt_with_children/average_percent


#достаем из таблицы процент должников и аффинити среди не имеющих детей 
per_debt_no_children=data_children[data_children['children']==0]['percent_of_debt'].values[0]
aff_debt_no_children=per_debt_no_children/average_percent

#печать
print('Процент задолженностей среди заемщиков с детьми: {:.1%}'.format(per_debt_with_children), ', aффинити {:.2f}'.format(aff_debt_with_children))
print('Процент задолженностей среди заемщиков без детей: {:.1%}'.format(per_debt_no_children), ', aффинити {:.2f}'.format(aff_debt_no_children))
data_with_children

Процент задолженностей среди заемщиков с детьми: 9.2% , aффинити 1.14
Процент задолженностей среди заемщиков без детей: 7.5% , aффинити 0.93


debt,children,children_group,0,1,percent_of_debt,affinity_of_debt
1,1,Y,4364.0,444.0,0.092346,1.137318
2,2,Y,1858.0,194.0,0.094542,1.164361
3,3,Y,303.0,27.0,0.081818,1.007658
4,4,Y,37.0,4.0,0.097561,1.201543
5,5,Y,9.0,NaN,NaN,NaN


### Вывод

Заемщики без детей реже задерживают выплаты по кредиту. Наличие задолженности не зависит от количества детей в семье.

### Влияние семейного положения на возврат кредита <a class="anchor" id="3.2"></a>

In [85]:
#создаем таблицу отражащую зависимость задолженности и семейного статуса
data_family=data.pivot_table(index='family_status',columns='debt', values='gender', aggfunc='count').reset_index()

#добавляем столбец с %% по задолженности для каждого сеймейного статуса
data_family['debt_percent']=data_family[1]/(data_family[0]+data_family[1])

#вычисляем аффинити
data_family['debt_affinity']=data_family['debt_percent']/average_percent

#сортируем по процентам по убыванию 
print(data_family.sort_values(by='debt_percent', ascending=False).reset_index(drop=True))

#выводы
print('\nНаибольший процент задолженностей по кредиту ({:.1%}) среди тех, кто'.format(data_family['debt_percent'].max()), data_family[data_family['debt_percent']==data_family['debt_percent'].max()]['family_status'].values[0])

print('\nНаименьший процент задолженностей по кредиту ({:.1%}) среди тех, кто'.format(data_family['debt_percent'].min()), data_family[data_family['debt_percent']==data_family['debt_percent'].min()]['family_status'].values[0])


debt          family_status      0    1  debt_percent  debt_affinity
0     не женат / не замужем   2523  273      0.097639       1.202510
1          гражданский брак   3749  385      0.093130       1.146974
2           женат / замужем  11334  927      0.075606       0.931145
3                 в разводе   1105   84      0.070648       0.870083
4            вдовец / вдова    888   63      0.066246       0.815875

Наибольший процент задолженностей по кредиту (9.8%) среди тех, кто не женат / не замужем

Наименьший процент задолженностей по кредиту (6.6%) среди тех, кто вдовец / вдова


### Вывод

те кто не женаты или в гражданском браке на 14-20% чаще задерживают выплаты по кредитам, безответственные люди

### Зависимость между уровнем дохода и возвратом кредита <a class="anchor" id="3.3"></a>

In [86]:
income_table=data[['debt','income_group','total_income']]
income_table.head()
debt_income=income_table.pivot_table(index='income_group',columns='debt', values='total_income', aggfunc='count').reset_index()
debt_income['percent_of_debts']=debt_income[1]/(debt_income[1]+debt_income[0])
debt_income['debts_affinity']=debt_income['percent_of_debts']/average_percent
debt_income[['income_group','percent_of_debts','debts_affinity']].sort_values(by='percent_of_debts', ascending=False)


debt,income_group,percent_of_debts,debts_affinity
4,C,0.088751,1.093038
3,B+,0.086225,1.061927
2,B,0.083831,1.032442
1,A+,0.079725,0.981884
5,C+,0.069725,0.858726
0,A,0.061995,0.763514


Самые богатые и самые бедные меньше всего склонны задолжать по кредиту, чаще всего задолжают заемщики с доходом 80-200К 

### Влияние целей кредита на его возврат   <a class="anchor" id="3.4"></a>

In [136]:
#создаем таблицу по целям и задолженностям
data_purpose=data[['main_purpose','debt']]

#группируем по целям и считаем для каждой цели сколько заемщиков и сколько из них имели задолженности
data_purpose_grouped=data_purpose.groupby('main_purpose').agg({'debt':['count','sum']})

#вычисляем процент задолженностей среди заемщиков по каждой цели
data_purpose_grouped['percent_of_debt']=data_purpose_grouped['debt']['sum']/data_purpose_grouped['debt']['count']

#вычисляем аффинити
data_purpose_grouped['debt_affinity']=data_purpose_grouped['percent_of_debt']/average_percent

#проводим сортировку по проценту
print(data_purpose_grouped.sort_values('percent_of_debt', ascending=False))


#исследуем по каким целям наиболее часто задерживают кредит
print('Всего:\n',data['main_purpose'].value_counts())
data_debted=data[data['debt']==1]
print('Задолженности:\n', data_debted['main_purpose'].value_counts())


               debt      percent_of_debt debt_affinity
              count  sum                              
main_purpose                                          
автомобиль     4279  400        0.093480      1.151280
образование    3988  369        0.092528      1.139553
свадьба        2313  183        0.079118      0.974403
недвижимость  10751  780        0.072551      0.893530
Всего:
 недвижимость    10751
автомобиль       4279
образование      3988
свадьба          2313
Name: main_purpose, dtype: int64
Задолженности:
 недвижимость    780
автомобиль      400
образование     369
свадьба         183
Name: main_purpose, dtype: int64


### Вывод

в целом кредиты на автомобиль и на образование чаще задерживают, а на недвижимость чаще погашают в срок.  Тем не менее недвижимость - наиболее популярная цель кредита, ожидаемо, что по ней и самый большой долг. 

## Шаг 4. Общий вывод <a class="anchor" id="4"></a>

Наибольее важными параметрами для оценки риска являются цель кредита, семейный статус и наличие детей.
Факторы риска: наличие детей, гражданский брак и цель кредита -автомобиль